<a href="https://colab.research.google.com/github/gooldan/vk_sentiment/blob/master/egor_test/sentiment_sutskever_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package libfuse2:amd64.
(Reading database ... 22280 files and directories currently installed.)
Preparing to unpack .../libfuse2_2.9.7-1ubuntu1_amd64.deb ...
Unpacking libfuse2:amd64 (2.9.7-1ubuntu1) ...
Selecting previously unselected package fuse.
Preparing to unpack .../fuse_2.9.7-1ubuntu1_amd64.deb ...
Unpacking fuse (2.9.7-1ubuntu1) ...
Selecting previously unselected package google-drive-ocamlfuse.
Preparing to unpack .../google-drive-ocamlfuse_0.7.0-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...
Setting up libfuse2:amd64 (2.9.7-1ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up fuse (2.9.7-1ubuntu1) ...
Setting up google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...


In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [3]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision

In [7]:
!pip install unidecode

    100% |████████████████████████████████| 235kB 5.6MB/s 


In [8]:
!git clone https://github.com/NVIDIA/sentiment-discovery.git

Cloning into 'sentiment-discovery'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 644 (delta 2), reused 4 (delta 2), pack-reused 638
Receiving objects: 100% (644/644), 53.71 MiB | 18.52 MiB/s, done.
Resolving deltas: 100% (351/351), done.
Checking out files: 100% (77/77), done.


In [0]:
import os
os.chdir('sentiment-discovery')

In [10]:
!ls

analysis	   data        generate.py	  model		setup.py
apex_utils	   data_utils  learning_rates.py  multiproc.py	transfer.py
classifier.py	   figures     LICENSE		  README.md
configure_data.py  fp16        main.py		  script_docs


In [11]:
!python setup.py install

torch.__version__  =  0.4.0
Building module.
running install
running bdist_egg
running egg_info
creating apex.egg-info
writing apex.egg-info/PKG-INFO
writing dependency_links to apex.egg-info/dependency_links.txt
writing requirements to apex.egg-info/requires.txt
writing top-level names to apex.egg-info/top_level.txt
writing manifest file 'apex.egg-info/SOURCES.txt'
reading manifest file 'apex.egg-info/SOURCES.txt'
writing manifest file 'apex.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/apex
copying apex/__init__.py -> build/lib/apex
creating build/lib/apex/reparameterization
copying apex/reparameterization/reparameterization.py -> build/lib/apex/reparameterization
copying apex/reparameterization/__init__.py -> build/lib/apex/reparameterization
copying apex/reparameterization/weight_norm.py -> build/lib/apex/reparameterization
creating build/lib/apex/utils
copying 

In [12]:
!pip install pydrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

def authenticate():
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    return drive

def upload_file_to_gdrive(file_name, drive=None):
    if drive is None:
        drive = authenticate()

    # Create & upload a file.
    uploaded = drive.CreateFile({'title': file_name})
    uploaded.SetContentFile(file_name)
    uploaded.Upload()
    print('Uploaded file with ID {}'.format(uploaded.get('id')))
    
    
def download_file_from_gdrive(file_name, id_file, drive=None):
    if drive is None:
        drive = authenticate()

    downloaded = drive.CreateFile({'id': id_file})
    # fetch content
    downloaded.FetchContent()
    # save content to file on the disk
    import shutil
    downloaded.content.seek(0)
    with open(file_name, 'wb') as f:
        shutil.copyfileobj(downloaded.content, f, length=131072)
    print("File `{}` was successfully loaded".format(file_name))

    100% |████████████████████████████████| 993kB 7.1MB/s 
  Running setup.py bdist_wheel for pydrive ... - \ done
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built pydrive


In [0]:
drive = authenticate()

In [14]:
# you can use these links to download files locally on your own laptop
files = {}
files['data1.json'] = 'https://drive.google.com/file/d/12Hs6KGUu7_Lrh_jqZ83wgScLHPPgc-92/view?usp=sharing'
# get file ids
import re

for key, val in files.items():
    fid = re.findall(r'https://drive.google.com/file/d/(.*?)/', val)[0]
    download_file_from_gdrive(file_name=key, id_file=fid, drive=drive)

File `data1.json` was successfully loaded


In [15]:
!ls

analysis	   data        fp16		  main.py	script_docs
apex_utils	   data1.json  generate.py	  model		setup.py
classifier.py	   data_utils  learning_rates.py  multiproc.py	transfer.py
configure_data.py  figures     LICENSE		  README.md


In [18]:
!python main.py --data data1.json --lazy --loose_json \
--text_key text --label_key label --num_shards 1002 \
--optim Adam --split 1000,1,1 --save_iters 1 --log-interval 1 --fp16

configuring data
Creating mlstm
* number of parameters: 86294784
FP16_Optimizer processing param group 0:
FP16_Optimizer received torch.cuda.HalfTensor with torch.Size([256, 64])
FP16_Optimizer received torch.cuda.HalfTensor with torch.Size([256, 4096])
FP16_Optimizer received torch.cuda.HalfTensor with torch.Size([256])
FP16_Optimizer received torch.cuda.HalfTensor with torch.Size([16384])
FP16_Optimizer received torch.cuda.HalfTensor with torch.Size([16384])
FP16_Optimizer received torch.cuda.HalfTensor with torch.Size([16384, 1])
FP16_Optimizer received torch.cuda.HalfTensor with torch.Size([16384, 64])
FP16_Optimizer received torch.cuda.HalfTensor with torch.Size([16384, 1])
FP16_Optimizer received torch.cuda.HalfTensor with torch.Size([16384, 4096])
FP16_Optimizer received torch.cuda.HalfTensor with torch.Size([4096, 1])
FP16_Optimizer received torch.cuda.HalfTensor with torch.Size([4096, 64])
FP16_Optimizer received torch.cuda.HalfTensor with torch.Size([4096, 1])
FP16_Optimizer 